# 演習：声質変換

## 環境構築

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/'My Drive'/vc_exercise2019

/content/drive/My Drive/vc_exercise2019


In [0]:
!pip3 install pyworld
!pip3 install pysptk
!pip3 install dtw

     |████████████████████████████████| 81kB 5.2MB/s 
  Created wheel for pyworld: filename=pyworld-0.2.8-cp36-cp36m-linux_x86_64.whl size=618600 sha256=c2c92020c83a79053686dd46c023dc20c7a749628b827425eb3a131ba430c8a5
  Stored in directory: /root/.cache/pip/wheels/c3/58/e5/a7e39ab92c56825f976970b97066753c68406c7fb0d80d4a4a
Successfully built pyworld
     |████████████████████████████████| 419kB 4.8MB/s 
  Created wheel for pysptk: filename=pysptk-0.1.17-cp36-cp36m-linux_x86_64.whl size=946772 sha256=1605b0ac063af017c901f5c6c91d273d6e676657c9788de774d83b08834e5fe6
  Stored in directory: /root/.cache/pip/wheels/eb/3a/b8/37bb86eec1b3dc9b6c4d7502ec532e3939ffc727eb0d24f2c9
Successfully built pysptk
  Created wheel for dtw: filename=dtw-1.3.3-cp36-none-any.whl size=5299 sha256=8904bc729cd9940793922e5ee8bb666fd71c41b6a970a7d2a72177969c8cddbc
  Stored in directory: /root/.cache/pip/wheels/a3/ec/0b/91460dfe9064731fe97be1d868ccaa23e0f99bfaea7c688f5a
Successfully built dtw


## 特徴量の分析

In [1]:
import os
import sys
import glob

from scipy.io import wavfile # for wavfile I/O
import pyworld as pw
import numpy as np
import pysptk as sptk

In [2]:
spklist = ["SF", "TF"]  # speaker list [source female speaker, target female speaker]
featlist = ["mgc","f0","ap"]

In [3]:
# Making directories for speech features
for s in spklist:
    for f in featlist:
        if not os.path.exists("data/{}/{}".format(s,f)):
            os.mkdir("data/{}/{}".format(s,f))

In [4]:
for s in spklist:
    wavlist = os.listdir("data/{}/wav".format(s))
    for wf in wavlist:
        # WORLD analysis for each file
        print("spekaer: {} file: {}".format(s,wf))
        fs, data = wavfile.read("data/{}/wav/{}".format(s,wf))
        data = data.astype(np.float)

        f0, t = pw.harvest(data, fs)
        sp = pw.cheaptrick(data, f0, t, fs)
        ap = pw.d4c(data, f0, t, fs)

        alpha = 0.42
        dim = 24
        mgc = sptk.sp2mc(sp, dim, alpha)

        bn, _ = os.path.splitext(wf)

        with open("data/{}/mgc/{}.mgc".format(s,bn),"wb") as f:
            mgc.tofile(f)
        with open("data/{}/f0/{}.f0".format(s,bn),"wb") as f:
            f0.tofile(f)
        with open("data/{}/ap/{}.ap".format(s,bn),"wb") as f:
            ap.tofile(f)

spekaer: SF file: atr503_a39.wav


/tmp/ipykernel_262311/1910573164.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = data.astype(np.float)


spekaer: SF file: atr503_a26.wav
spekaer: SF file: atr503_a32.wav
spekaer: SF file: atr503_a10.wav
spekaer: SF file: atr503_a08.wav
spekaer: SF file: atr503_a34.wav
spekaer: SF file: atr503_a30.wav
spekaer: SF file: atr503_a16.wav
spekaer: SF file: atr503_a27.wav
spekaer: SF file: atr503_a49.wav
spekaer: SF file: atr503_a02.wav
spekaer: SF file: atr503_a36.wav
spekaer: SF file: atr503_a15.wav
spekaer: SF file: atr503_a13.wav
spekaer: SF file: atr503_a03.wav
spekaer: SF file: atr503_a45.wav
spekaer: SF file: atr503_a04.wav
spekaer: SF file: atr503_a43.wav
spekaer: SF file: atr503_a40.wav
spekaer: SF file: atr503_a50.wav
spekaer: SF file: atr503_a22.wav
spekaer: SF file: atr503_a23.wav
spekaer: SF file: atr503_a48.wav
spekaer: SF file: atr503_a47.wav
spekaer: SF file: atr503_a09.wav
spekaer: SF file: atr503_a20.wav
spekaer: SF file: atr503_a05.wav
spekaer: SF file: atr503_a41.wav
spekaer: SF file: atr503_a17.wav
spekaer: SF file: atr503_a19.wav
spekaer: SF file: atr503_a11.wav
spekaer: S

## フレーム毎時間アラインメント

In [21]:
import os
import sys
import array

from fastdtw import fastdtw
import numpy as np
import pysptk as sptk

In [22]:
srcspk = "SF"
tgtspk = "TF"

mgclist = os.listdir("data/{}/mgc".format(srcspk))

if not os.path.isdir("data/{}/data".format(srcspk)):
    os.mkdir("data/{}/data".format(srcspk))
if not os.path.isdir("data/{}/data".format(tgtspk)):
    os.mkdir("data/{}/data".format(tgtspk))

In [23]:
def distfunc(x,y):
    # Euclid distance except first dim
    return np.linalg.norm(x[1:]-y[1:])

In [35]:
dim = 25 # mgc dim + 1
for mf in mgclist:
    print(mf)
    bn, _ = os.path.splitext(mf)
    srcfile = "data/{}/mgc/{}".format(srcspk,mf)
    tgtfile = "data/{}/mgc/{}".format(tgtspk,mf)

    with open(srcfile,"rb") as f:
        x = np.fromfile(f, dtype="<f8", sep="")
        x = x.reshape(len(x)//dim,dim)
    with open(tgtfile,"rb") as f:
        y = np.fromfile(f, dtype="<f8", sep="")
        y = y.reshape(len(y)//dim,dim)
    print("framelen: (x,y) = {} {}".format(len(x),len(y)))
    _,twf = fastdtw(x,y,dist=distfunc)
    srcout = "data/{}/data/{}.dat".format(srcspk,bn)
    tgtout = "data/{}/data/{}.dat".format(tgtspk,bn)
    twfx = list(map(lambda l: l[0], twf))
    twfy = list(map(lambda l: l[1], twf))
    
    with open(srcout,"wb") as f:
        x[twfx].tofile(f)
    with open(tgtout,"wb") as f:
        y[twfy].tofile(f)

atr503_a33.mgc
framelen: (x,y) = 1951 1943
atr503_a26.mgc
framelen: (x,y) = 1461 1455
atr503_a34.mgc
framelen: (x,y) = 1633 1573
atr503_a46.mgc
framelen: (x,y) = 1881 1993
atr503_a47.mgc
framelen: (x,y) = 1297 1123
atr503_a22.mgc
framelen: (x,y) = 1003 1035
atr503_a10.mgc
framelen: (x,y) = 881 891
atr503_a15.mgc
framelen: (x,y) = 1635 1605
atr503_a21.mgc
framelen: (x,y) = 1123 969
atr503_a44.mgc
framelen: (x,y) = 1151 1235
atr503_a20.mgc
framelen: (x,y) = 1041 1015
atr503_a16.mgc
framelen: (x,y) = 1183 1205
atr503_a40.mgc
framelen: (x,y) = 1301 1429
atr503_a30.mgc
framelen: (x,y) = 1501 1529
atr503_a41.mgc
framelen: (x,y) = 913 1109
atr503_a29.mgc
framelen: (x,y) = 1611 1883
atr503_a18.mgc
framelen: (x,y) = 1349 1217
atr503_a03.mgc
framelen: (x,y) = 1027 907
atr503_a37.mgc
framelen: (x,y) = 1137 1265
atr503_a11.mgc
framelen: (x,y) = 1165 1115
atr503_a08.mgc
framelen: (x,y) = 1153 1039
atr503_a50.mgc
framelen: (x,y) = 1587 1755
atr503_a43.mgc
framelen: (x,y) = 1489 1657
atr503_a35.mgc
f

## 音声変換モデルの学習

In [36]:
# Listing training/evaluation data
!mkdir -p conf
!ls data/SF/data/ | head -45 | sed -e 's/\.dat//' > conf/train.list
!ls data/SF/data/ | tail -5 | sed -e 's/\.dat//' > conf/eval.list

In [37]:
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
import os
import sys
import time

In [38]:
def get_dataset(dim=25):
    x = []
    y = []
    datalist = []
    with open("conf/train.list","r") as f:
        for line in f:
            line = line.rstrip()
            datalist.append(line)

    for d in datalist:
        print(d)
        with open("data/SF/data/{}.dat".format(d),"rb") as f:
            dat = np.fromfile(f,dtype="<f8",sep="")
            x.append(dat.reshape(len(dat)//dim,dim))
        with open("data/TF/data/{}.dat".format(d),"rb") as f:
            dat = np.fromfile(f,dtype="<f8",sep="")
            y.append(dat.reshape(len(dat)//dim,dim))
    return x,y

In [39]:
class VCDNN(nn.Module):
        def __init__(self, dim=25, n_units=256):
            super(VCDNN, self).__init__()
            self.fc = nn.ModuleList([
                           nn.Linear(dim, n_units),
                           nn.Linear(n_units, n_units),
                           nn.Linear(n_units, dim)
            ])
            
        def forward(self, x):
            h1 = F.relu(self.fc[0](x))
            h2 = F.relu(self.fc[1](h1))
            h3 = self.fc[2](h2)
            return h3
        
        def get_predata(self, x):
            _x = torch.from_numpy(x.astype(np.float32))
            return self.forward(_x).detach().numpy()

In [40]:
x_train, y_train = get_dataset()
# parameters for training
n_epoch = 50
dim = 25
n_units = 128
N = len(x_train)

model = VCDNN(dim,n_units)
model.double()
optimizer = optim.Adam(model.parameters())

loss_fn = nn.MSELoss()

# loop
model.train()

losses = []
sum_loss = 0

for epoch in range(1, n_epoch + 1):
    sum_loss = 0

    for i in range(0, N):
        x_batch =torch.from_numpy(x_train[i])
        y_batch = torch.from_numpy(y_train[i])
        
        optimizer.zero_grad()
        
        predict_y_batch = model(x_batch)
        loss = loss_fn(predict_y_batch, y_batch)
        loss.backward()
        optimizer.step()
        sum_loss += loss.item()
        
        average_loss = sum_loss / N
        losses.append(average_loss)

        print("epoch: {}/{}  loss: {}".format(epoch, n_epoch, average_loss))

if not os.path.isdir("model"):
    os.mkdir("model")
torch.save(model.state_dict(), "model/vcmodel.model")


atr503_a01
atr503_a02
atr503_a03
atr503_a04
atr503_a05
atr503_a06
atr503_a07
atr503_a08
atr503_a09
atr503_a10
atr503_a11
atr503_a12
atr503_a13
atr503_a14
atr503_a15
atr503_a16
atr503_a17
atr503_a18
atr503_a19
atr503_a20
atr503_a21
atr503_a22
atr503_a23
atr503_a24
atr503_a25
atr503_a26
atr503_a27
atr503_a28
atr503_a29
atr503_a30
atr503_a31
atr503_a32
atr503_a33
atr503_a34
atr503_a35
atr503_a36
atr503_a37
atr503_a38
atr503_a39
atr503_a40
atr503_a41
atr503_a42
atr503_a43
atr503_a44
atr503_a45
epoch: 1/50  loss: 0.01924231811206356
epoch: 1/50  loss: 0.033682974432136456
epoch: 1/50  loss: 0.04800194766197621
epoch: 1/50  loss: 0.06376625111981314
epoch: 1/50  loss: 0.07710653227011333
epoch: 1/50  loss: 0.08912130754911875
epoch: 1/50  loss: 0.10252079280957359
epoch: 1/50  loss: 0.1128678499398528
epoch: 1/50  loss: 0.12365898849542685
epoch: 1/50  loss: 0.1329145361355178
epoch: 1/50  loss: 0.14225587887076915
epoch: 1/50  loss: 0.1503421712721148
epoch: 1/50  loss: 0.1558638231971963
e

epoch: 5/50  loss: 0.025430665431390338
epoch: 5/50  loss: 0.026611757005334198
epoch: 5/50  loss: 0.02762453048463653
epoch: 5/50  loss: 0.028848734160488852
epoch: 5/50  loss: 0.029806330023895225
epoch: 5/50  loss: 0.030867255064882623
epoch: 5/50  loss: 0.031779791765859766
epoch: 5/50  loss: 0.03275308848158563
epoch: 5/50  loss: 0.03392887923191081
epoch: 5/50  loss: 0.034789620433064584
epoch: 5/50  loss: 0.03561589342982786
epoch: 5/50  loss: 0.03651348638892057
epoch: 5/50  loss: 0.03785765549966912
epoch: 5/50  loss: 0.03913455958958607
epoch: 5/50  loss: 0.0402529134820313
epoch: 5/50  loss: 0.041312848450692365
epoch: 5/50  loss: 0.04217029997262325
epoch: 5/50  loss: 0.04319006619224709
epoch: 5/50  loss: 0.04412460822672278
epoch: 5/50  loss: 0.045222508871914306
epoch: 5/50  loss: 0.04639051428758478
epoch: 5/50  loss: 0.04751214061769768
epoch: 6/50  loss: 0.0012674245804578266
epoch: 6/50  loss: 0.002316740861917539
epoch: 6/50  loss: 0.00341236716982525
epoch: 6/50  l

epoch: 10/50  loss: 0.011898024008063815
epoch: 10/50  loss: 0.012843271216547528
epoch: 10/50  loss: 0.014052636164784969
epoch: 10/50  loss: 0.014979289069903699
epoch: 10/50  loss: 0.016020386321556408
epoch: 10/50  loss: 0.017119171529030943
epoch: 10/50  loss: 0.01792862288175683
epoch: 10/50  loss: 0.01886332101285709
epoch: 10/50  loss: 0.019807316073689088
epoch: 10/50  loss: 0.02084608302266955
epoch: 10/50  loss: 0.021852022753714975
epoch: 10/50  loss: 0.022900873107048744
epoch: 10/50  loss: 0.023779657837203445
epoch: 10/50  loss: 0.024827712479656903
epoch: 10/50  loss: 0.025770667485849028
epoch: 10/50  loss: 0.026937710314982467
epoch: 10/50  loss: 0.027804567122244873
epoch: 10/50  loss: 0.028751918457177613
epoch: 10/50  loss: 0.0296396574558694
epoch: 10/50  loss: 0.030555436672828885
epoch: 10/50  loss: 0.031636322783704915
epoch: 10/50  loss: 0.0324317387308747
epoch: 10/50  loss: 0.03317235537325249
epoch: 10/50  loss: 0.0340311103446765
epoch: 10/50  loss: 0.0353

epoch: 15/50  loss: 0.014591644372770763
epoch: 15/50  loss: 0.015590585456389914
epoch: 15/50  loss: 0.016655044148249202
epoch: 15/50  loss: 0.017443497268343315
epoch: 15/50  loss: 0.0183381302722146
epoch: 15/50  loss: 0.01926450969859185
epoch: 15/50  loss: 0.0202920654197262
epoch: 15/50  loss: 0.021281465855998858
epoch: 15/50  loss: 0.02230434967701298
epoch: 15/50  loss: 0.023166408015454904
epoch: 15/50  loss: 0.024136889486106223
epoch: 15/50  loss: 0.025050610575403667
epoch: 15/50  loss: 0.02618890095213985
epoch: 15/50  loss: 0.027028581295372625
epoch: 15/50  loss: 0.027931360362893183
epoch: 15/50  loss: 0.028809381474922333
epoch: 15/50  loss: 0.029705650876675243
epoch: 15/50  loss: 0.03074212340288937
epoch: 15/50  loss: 0.031517217162318804
epoch: 15/50  loss: 0.03223383040818071
epoch: 15/50  loss: 0.03308424242151492
epoch: 15/50  loss: 0.034339613076608565
epoch: 15/50  loss: 0.03550168896709646
epoch: 15/50  loss: 0.03651431788889914
epoch: 15/50  loss: 0.037500

epoch: 20/50  loss: 0.004140619496382665
epoch: 20/50  loss: 0.005007150878075441
epoch: 20/50  loss: 0.0058484063124382
epoch: 20/50  loss: 0.006883424967806879
epoch: 20/50  loss: 0.007727365543522185
epoch: 20/50  loss: 0.008635703500067737
epoch: 20/50  loss: 0.00958498579158562
epoch: 20/50  loss: 0.0104088322935828
epoch: 20/50  loss: 0.011435329875243093
epoch: 20/50  loss: 0.01234123870695792
epoch: 20/50  loss: 0.01351977450284934
epoch: 20/50  loss: 0.014402859154135021
epoch: 20/50  loss: 0.01537871773164882
epoch: 20/50  loss: 0.016427251159875266
epoch: 20/50  loss: 0.01720842242762189
epoch: 20/50  loss: 0.018085793808403697
epoch: 20/50  loss: 0.019010369181387175
epoch: 20/50  loss: 0.020038091282103015
epoch: 20/50  loss: 0.021020493946538144
epoch: 20/50  loss: 0.022036096254007757
epoch: 20/50  loss: 0.02289227337877094
epoch: 20/50  loss: 0.023828761465251657
epoch: 20/50  loss: 0.02472345776712502
epoch: 20/50  loss: 0.025847567996841292
epoch: 20/50  loss: 0.02667

...

epoch: 29/50  loss: 0.005812128473400583
epoch: 29/50  loss: 0.006820090917560564
epoch: 29/50  loss: 0.007643556195954932
epoch: 29/50  loss: 0.008533264452297469
epoch: 29/50  loss: 0.00947693404707869
epoch: 29/50  loss: 0.010308920468645582
epoch: 29/50  loss: 0.01134033510438756
epoch: 29/50  loss: 0.012235504810331786
epoch: 29/50  loss: 0.013401148186909125
epoch: 29/50  loss: 0.014259839858508913
epoch: 29/50  loss: 0.015207085703879741
epoch: 29/50  loss: 0.016241076212088725
epoch: 29/50  loss: 0.017008525608502262
epoch: 29/50  loss: 0.01786285745594782
epoch: 29/50  loss: 0.018787790922556814
epoch: 29/50  loss: 0.019817410384714903
epoch: 29/50  loss: 0.020791379639110978
epoch: 29/50  loss: 0.021807147986802247
epoch: 29/50  loss: 0.02266464856853798
epoch: 29/50  loss: 0.02357349732067998
epoch: 29/50  loss: 0.024439872419794714
epoch: 29/50  loss: 0.025543627847715817
epoch: 29/50  loss: 0.026353703726791026
epoch: 29/50  loss: 0.027228283271555437
epoch: 29/50  loss: 0

epoch: 33/50  loss: 0.040948456093494796
epoch: 33/50  loss: 0.04195232883107922
epoch: 34/50  loss: 0.0011716287055793596
epoch: 34/50  loss: 0.0021271051536301323
epoch: 34/50  loss: 0.003129988838207074
epoch: 34/50  loss: 0.0041269053058478325
epoch: 34/50  loss: 0.004978468719337015
epoch: 34/50  loss: 0.005805409574259784
epoch: 34/50  loss: 0.006800629059359505
epoch: 34/50  loss: 0.007612650821163128
epoch: 34/50  loss: 0.008490718378350845
epoch: 34/50  loss: 0.009425518716523753
epoch: 34/50  loss: 0.010256733463824053
epoch: 34/50  loss: 0.011288783501874981
epoch: 34/50  loss: 0.012183341992681099
epoch: 34/50  loss: 0.013347719558425375
epoch: 34/50  loss: 0.014197872513746016
epoch: 34/50  loss: 0.01513393024618519
epoch: 34/50  loss: 0.01616098250638118
epoch: 34/50  loss: 0.01691928530618393
epoch: 34/50  loss: 0.017758350692271288
epoch: 34/50  loss: 0.018677329871363123
epoch: 34/50  loss: 0.01970386623017043
epoch: 34/50  loss: 0.02067285125992779
epoch: 34/50  loss:

epoch: 38/50  loss: 0.03113353464748351
epoch: 38/50  loss: 0.03198052469183301
epoch: 38/50  loss: 0.033213391424025906
epoch: 38/50  loss: 0.03430267532287571
epoch: 38/50  loss: 0.035289165800157354
epoch: 38/50  loss: 0.03623114059704254
epoch: 38/50  loss: 0.036978738084192775
epoch: 38/50  loss: 0.0378821754928797
epoch: 38/50  loss: 0.0387281068059341
epoch: 38/50  loss: 0.039718376688892686
epoch: 38/50  loss: 0.04073638020874079
epoch: 38/50  loss: 0.04173287641735539
epoch: 39/50  loss: 0.001170301823081657
epoch: 39/50  loss: 0.0021269704740928025
epoch: 39/50  loss: 0.0031306659418647285
epoch: 39/50  loss: 0.004126318470093216
epoch: 39/50  loss: 0.004975294471707329
epoch: 39/50  loss: 0.0057996634775331
epoch: 39/50  loss: 0.006784218740347482
epoch: 39/50  loss: 0.007587985396945666
epoch: 39/50  loss: 0.00845604037756109
epoch: 39/50  loss: 0.009382357657213968
epoch: 39/50  loss: 0.010209902546651184
epoch: 39/50  loss: 0.011239830251440977
epoch: 39/50  loss: 0.01213

epoch: 43/50  loss: 0.02516352886534164
epoch: 43/50  loss: 0.02595802055325038
epoch: 43/50  loss: 0.02681941985082216
epoch: 43/50  loss: 0.02768554529070747
epoch: 43/50  loss: 0.02855888513494965
epoch: 43/50  loss: 0.02953302505715532
epoch: 43/50  loss: 0.03027834645936955
epoch: 43/50  loss: 0.03098083626000315
epoch: 43/50  loss: 0.03182464869034921
epoch: 43/50  loss: 0.03305407186378963
epoch: 43/50  loss: 0.03413718610778625
epoch: 43/50  loss: 0.035123533845295415
epoch: 43/50  loss: 0.036060272555160124
epoch: 43/50  loss: 0.03680465132418073
epoch: 43/50  loss: 0.037701679968642134
epoch: 43/50  loss: 0.03854144847621416
epoch: 43/50  loss: 0.03952573010836082
epoch: 43/50  loss: 0.040535284468464834
epoch: 43/50  loss: 0.041525441893876476
epoch: 44/50  loss: 0.001167923480825925
epoch: 44/50  loss: 0.002123732725385739
epoch: 44/50  loss: 0.0031264301074963867
epoch: 44/50  loss: 0.004120056130623939
epoch: 44/50  loss: 0.00496709176959316
epoch: 44/50  loss: 0.00578940

epoch: 48/50  loss: 0.011154107772043636
epoch: 48/50  loss: 0.012046516925131494
epoch: 48/50  loss: 0.013200964192157227
epoch: 48/50  loss: 0.014034684430430416
epoch: 48/50  loss: 0.014955409345720757
epoch: 48/50  loss: 0.015971584787077296
epoch: 48/50  loss: 0.016712476275825193
epoch: 48/50  loss: 0.017518486603414908
epoch: 48/50  loss: 0.01842444781143269
epoch: 48/50  loss: 0.019438864386552623
epoch: 48/50  loss: 0.020392239942715214
epoch: 48/50  loss: 0.02138935774308499
epoch: 48/50  loss: 0.02225139816040461
epoch: 48/50  loss: 0.023132347261139097
epoch: 48/50  loss: 0.02397110493645072
epoch: 48/50  loss: 0.02505659631024273
epoch: 48/50  loss: 0.025848395383434985
epoch: 48/50  loss: 0.026707319491193524
epoch: 48/50  loss: 0.02757183340318109
epoch: 48/50  loss: 0.02844123680814329
epoch: 48/50  loss: 0.029407592969000623
epoch: 48/50  loss: 0.0301493568024125
epoch: 48/50  loss: 0.03084938281182635
epoch: 48/50  loss: 0.03168952671292203
epoch: 48/50  loss: 0.03291

In [41]:
!ls ./model/

vcmodel.model


## 学習したモデルによる音声の変換

In [42]:
import numpy as np
import pysptk as sptk
import pyworld as pw
from scipy.io import wavfile
import os
import sys
import time

In [43]:
dim = 25
n_units = 128

model = VCDNN(dim,n_units)
_ = model.load_state_dict(torch.load("model/vcmodel.model"))

In [44]:
# test data
x = []
datalist = []
with open("conf/eval.list","r") as f:
    for line in f:
        line = line.rstrip()
        datalist.append(line)

for d in datalist:
    with open("data/SF/mgc/{}.mgc".format(d),"rb") as f:
        dat = np.fromfile(f,dtype="<f8",sep="")
        x.append(dat.reshape(len(dat)//dim,dim))

if not os.path.isdir("result"):
    os.mkdir("result")
if not os.path.isdir("result/wav"):
    os.mkdir("result/wav")

fs = 16000
fftlen = 512
alpha = 0.42
for i in range(0,len(datalist)):
    outfile = "result/wav/{}.wav".format(datalist[i])
    with open("data/SF/f0/{}.f0".format(datalist[i]),"rb") as f:
        f0 = np.fromfile(f, dtype="<f8", sep="")
    with open("data/SF/ap/{}.ap".format(datalist[i]),"rb") as f:
        ap = np.fromfile(f, dtype="<f8", sep="")
        ap = ap.reshape(len(ap)//(fftlen+1),fftlen+1)
    y = model.get_predata(x[i])
    y = y.astype(np.float64)
    sp = sptk.mc2sp(y, alpha, fftlen*2)
    owav = pw.synthesize(f0, sp, ap, fs)
    owav = np.clip(owav, -32768, 32767)
    wavfile.write(outfile, fs, owav.astype(np.int16))

In [45]:
!ls result/wav

atr503_a46.wav	atr503_a47.wav	atr503_a48.wav	atr503_a49.wav	atr503_a50.wav
